In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -r "/content/drive/My Drive/Colab Notebooks/reformer-language-model/requirements.txt"

     |████████████████████████████████| 1.9MB 5.8MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 307kB 14.8MB/s 
     |████████████████████████████████| 3.2MB 18.9MB/s 
     |████████████████████████████████| 890kB 50.6MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp37-cp37m-linux_x86_64.whl size=2086030 sha256=05c4c94cd344facd1140f40f5f1a20eb4b8c8f47445ef23436edb52777932f11
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=9beb364782b99d061e573e0cdcac1c6509b5fef468713b4c40e8f3e4bd182165
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for product-key-memory: filename=product_key_memory-0.1.10-cp37-none-any.whl size=3072 sha256=ba372ad2259307a1cb2

## 필요 패키지 선언

In [3]:
from __future__ import absolute_import, division, print_function
# import warnings
# warnings.filterwarnings("ignore")

import argparse
import logging
import os
import random
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/reformer-language-model')
from io import open
import json

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler,SequentialSampler, TensorDataset)
from tqdm import tqdm, trange,tqdm_notebook

from reformer_pytorch import ReformerLM
from model.mrc import ReformerMRCModel
from transformers.optimization import AdamW
from util.schedule import WarmupLinearSchedule
from transformers import BertTokenizer
from util.arg import ModelConfig
from util.korquad_utils import (read_squad_examples, convert_examples_to_features, RawResult, write_predictions,evaluate)

if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

## 파일경로 및 Hyper-parameters 초기화

#### PATH

##### google drive path

In [4]:
gdrive_path = "/content/drive/My Drive/Colab Notebooks/reformer-language-model"

##### korquad data

In [5]:
train_file = f"{gdrive_path}/data/korquad/KorQuAD_v1.0_train.json"
dev_file = f"{gdrive_path}/data/korquad/KorQuAD_v1.0_dev.json"

In [6]:
print(dev_file)
print(gdrive_path)

/content/drive/My Drive/Colab Notebooks/reformer-language-model/data/korquad/KorQuAD_v1.0_dev.json
/content/drive/My Drive/Colab Notebooks/reformer-language-model


In [7]:
output_dir = f'{gdrive_path}/korquad'
checkpoint_path = os.path.join(gdrive_path, "checkpoints/reformer-mlm-small.pth") # epoch27-reformer-small.pt
# checkpoint_path = os.path.join(gdrive_path, "checkpoints/epoch27-reformer-small.pt")
config_path = os.path.join(gdrive_path, "config/mlm/mlm-small-pretrain.json")
model_name = "reformer-mlm-small"

#### Hyper Parameteer

In [8]:
doc_stride = 128 
max_query_length = 96 
max_answer_length = 30 
n_best_size = 20 

train_batch_size = 64 
learning_rate = 2e-4
warmup_proportion = 0.1 
num_train_epochs = 10.0

max_grad_norm = 1.0
adam_epsilon = 1e-6
weight_decay = 0.01

#### Device 설정

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
logger.info("device: {} n_gpu: {}".format(device, n_gpu))

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if n_gpu > 0:
        torch.cuda.manual_seed_all(SEED)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

03/07/2021 02:56:49 - INFO - __main__ -   device: cuda n_gpu: 1


#### Eval Method KorQuAD

In [10]:
def eval(model, epoch, eval_examples,eval_features,predict_batch_size):
  predict = dev_file

  all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
  all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
  all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
  all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
  dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_example_index)
  sampler = SequentialSampler(dataset)
  dataloader = DataLoader(dataset, sampler=sampler, batch_size=predict_batch_size)

  logger.info("***** Evaluating *****")
  logger.info("  Num features = %d", len(dataset))
  logger.info("  Batch size = %d", predict_batch_size)

  model.eval()
  model.to(device)
  all_results = []
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  if n_gpu > 0:
    torch.cuda.manual_seed_all(SEED)

  logger.info("Start evaluating!")
  for input_ids, input_mask, segment_ids, example_indices in dataloader: #tqdm(dataloader, desc="Evaluating", leave=True, position=1):
    input_ids = input_ids.to(device)
    with torch.no_grad():
      batch_start_logits, batch_end_logits = model(input_ids)
    for i, example_index in enumerate(example_indices):
      start_logits = batch_start_logits[i].detach().cpu().tolist()
      end_logits = batch_end_logits[i].detach().cpu().tolist()
      eval_feature = eval_features[example_index.item()]
      unique_id = int(eval_feature.unique_id)
      all_results.append(RawResult(unique_id=unique_id,
                                   start_logits=start_logits,
                                   end_logits=end_logits))
  output_prediction_file = os.path.join(output_dir, f"{model_name}_predictions_{epoch}.json")
  output_nbest_file = os.path.join(output_dir, f"{model_name}_nbest_predictions_{epoch}.json")
  write_predictions(eval_examples, eval_features, all_results,
                    n_best_size, max_answer_length,
                    False, output_prediction_file, output_nbest_file,
                    None, False, False, 0.0)

  with open(predict) as dataset_file:
    dataset_json = json.load(dataset_file)
    dataset = dataset_json['data']

  with open(os.path.join(output_prediction_file)) as prediction_file:
    logger.info(f"Evaluating File: {output_prediction_file}")
    predictions = json.load(prediction_file)
  logger.info(json.dumps(evaluate(dataset, predictions)))

### Model Load

In [11]:
config = ModelConfig(config_path).get_config()

# Tokenizer
tokenizer = BertTokenizer(vocab_file=config.vocab_path, do_lower_case=False)

# 2. Model Hyperparameter
batch_size = 128
max_seq_length = config.max_seq_len
# 3. Prepare model
model = ReformerMRCModel(
    num_tokens=tokenizer.vocab_size,
    dim=config.dim,
    depth=config.depth,
    heads=config.n_head,
    max_seq_len= max_seq_length,
    causal=False  # auto-regressive 학습을 위한 설정
)

##### Eval Data

In [12]:
eval_examples = read_squad_examples(input_file=dev_file,
                                    is_training=False,
                                    version_2_with_negative=False)
eval_features = convert_examples_to_features(examples=eval_examples,
                                              tokenizer=tokenizer,
                                              max_seq_length=config.max_seq_len,
                                              doc_stride=doc_stride,
                                              max_query_length=max_query_length,
                                              is_training=False)

03/07/2021 02:56:51 - INFO - util.korquad_utils -   *** Example ***
03/07/2021 02:56:51 - INFO - util.korquad_utils -   unique_id: 1000000000
03/07/2021 02:56:51 - INFO - util.korquad_utils -   example_index: 0
03/07/2021 02:56:51 - INFO - util.korquad_utils -   doc_span_index: 0
03/07/2021 02:56:51 - INFO - util.korquad_utils -   tokens: [CLS] 임 ##종 ##석이 여의도 농민 폭력 시위 ##를 주도 ##한 혐의로 지명 ##수 ##배 된 날 ##은 ? [SEP] 1989년 2월 15일 여의도 농민 폭력 시위 ##를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처 ##벌 ##에 ##관 ##한 ##법 ##률 ##위 ##반 ) 으로 지명 ##수 ##배 ##되었다 . 1989년 3월 12일 서울 ##지방 ##검 ##찰 ##청 공 ##안 ##부는 임 ##종 ##석의 사전 ##구 ##속 ##영 ##장을 발 ##부 ##받았다 . 같은 해 6월 30일 평양 ##축 ##전에 임 ##수 ##경을 대표 ##로 파견 ##하여 국가 ##보 ##안 ##법 ##위 ##반 혐의 ##가 추가 ##되었다 . 경찰은 12월 18일 ~ 20일 사이 서울 경희 ##대학교 ##에서 임 ##종 ##석이 성명 발표 ##를 추진하고 있다는 첩 ##보를 입수 ##했고 , 12월 18일 오전 7시 40 ##분 경 가스 ##총 ##과 전자 ##봉 ##으로 무장 ##한 특 ##공 ##조 및 대공 ##과 직원 12 ##명 등 22 ##명의 사 ##복 경찰 ##을 승 ##용차 8 ##대에 나누어 경희 ##대학교 ##에 투입 ##했다 . 1989년 12월 18일 오전 8시 15 ##분 경 서울 ##청 ##량 ##리 ##경찰 ##서는 호 ##위 학생

In [13]:
# 시작 Epoch
start_epoch = 0
 
# Reformer MLM 모델
if os.path.isfile(checkpoint_path):
  checkpoint = torch.load(checkpoint_path, map_location=device)
  model.reformer.load_state_dict(checkpoint['model_state_dict'],strict=False)
  logger.info(f'Load Reformer Model')

03/07/2021 02:57:35 - INFO - __main__ -   Load Reformer Model


### Train Koquad

In [14]:



num_params = count_parameters(model)
logger.info("Total Parameter: %d" % num_params)
model.to(device)

cached_train_features_file = train_file + '_{0}_{1}_{2}'.format(str(max_seq_length), str(doc_stride),
                                                                      str(max_query_length))
train_examples = read_squad_examples(input_file=train_file, is_training=True, version_2_with_negative=False)
try:
    with open(cached_train_features_file, "rb") as reader:
        train_features = pickle.load(reader)
except:
    train_features = convert_examples_to_features(
        examples=train_examples,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        doc_stride=doc_stride,
        max_query_length=max_query_length,
        is_training=True)
    logger.info("  Saving train features into cached file %s", cached_train_features_file)
    with open(cached_train_features_file, "wb") as writer:
        pickle.dump(train_features, writer)

num_train_optimization_steps = int(len(train_features) / train_batch_size) * num_train_epochs

# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
      'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters,
                  lr=learning_rate,
                  eps=adam_epsilon)
scheduler = WarmupLinearSchedule(optimizer,
                                  warmup_steps=num_train_optimization_steps*0.1,
                                  t_total=num_train_optimization_steps)

logger.info("***** Running training *****")
logger.info("  Num orig examples = %d", len(train_examples))
logger.info("  Num split examples = %d", len(train_features))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_optimization_steps)
num_train_step = num_train_optimization_steps

all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_start_positions = torch.tensor([f.start_position for f in train_features], dtype=torch.long)
all_end_positions = torch.tensor([f.end_position for f in train_features], dtype=torch.long)
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                            all_start_positions, all_end_positions)

train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

model.train()
global_step = 0
epoch = start_epoch
for i in range(start_epoch, int(num_train_epochs)):
    iter_bar = tqdm(train_dataloader, desc=f"Epoch-{i} Train(XX Epoch) Step(XX/XX) (Mean loss=X.X) (loss=X.X)")
    tr_step, total_loss, mean_loss = 0, 0., 0.
    for step, batch in enumerate(iter_bar):
        if n_gpu == 1:
            batch = tuple(t.to(device) for t in batch)  # multi-gpu does scattering it-self
        
        input_ids, input_mask, segment_ids, start_positions, end_positions = batch

        loss = model(input_ids, start_positions, end_positions)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        scheduler.step()
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1
        tr_step += 1
        total_loss += loss.item()
        mean_loss = total_loss / tr_step
        iter_bar.set_description(f"Epoch-{i} Train Step(%d / %d) (Mean loss=%5.5f) (loss=%5.5f)" %
                                  (global_step, num_train_step, mean_loss, loss.item()))

    logger.info("** ** * Saving file * ** **")
    model_checkpoint = f"{model_name}_{epoch}.bin"
    logger.info(model_checkpoint)
    output_model_file = os.path.join(output_dir, model_checkpoint)
    # 평가
    eval(model, epoch, eval_examples,eval_features,train_batch_size)
    model.train()
    
    torch.save(model.state_dict(), output_model_file)
    epoch += 1


03/07/2021 02:57:35 - INFO - __main__ -   Total Parameter: 28896770
03/07/2021 02:57:55 - INFO - __main__ -   ***** Running training *****
03/07/2021 02:57:55 - INFO - __main__ -     Num orig examples = 60407
03/07/2021 02:57:55 - INFO - __main__ -     Num split examples = 62555
03/07/2021 02:57:55 - INFO - __main__ -     Batch size = 64
03/07/2021 02:57:55 - INFO - __main__ -     Num steps = 9770
Epoch-0 Train(XX Epoch) Step(XX/XX) (Mean loss=X.X) (loss=X.X):   0%|          | 0/978 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable

## Evaluation

In [15]:
# electra_korquad_15.bin
num =3
checkpoint = os.path.join(gdrive_path, f"korquad/reformer-mlm-base_3.bin")
predict = os.path.join(gdrive_path, "data/korquad/KorQuAD_v1.0_tra.json")
prediction_name = f"predictions_{num}.json"
nbest_name = f"nbest_predictions_{num}.json"

predict_batch_size = 16 #@param {type: "integer"}

### Model load

In [16]:
model.load_state_dict(torch.load(checkpoint))
num_params = count_parameters(model)
logger.info("Total Parameter: %d" % num_params)

RuntimeError: ignored

In [ ]:
eval_examples = read_squad_examples(input_file=predict,
                               is_training=False,
                               version_2_with_negative=False)
eval_features = convert_examples_to_features(examples=eval_examples,
                                        tokenizer=tokenizer,
                                        max_seq_length=max_seq_length,
                                        doc_stride=doc_stride,
                                        max_query_length=max_query_length,
                                        is_training=False)

In [ ]:
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_example_index)
sampler = SequentialSampler(dataset)
dataloader = DataLoader(dataset, sampler=sampler, batch_size=predict_batch_size)

logger.info("***** Evaluating *****")
logger.info("  Num features = %d", len(dataset))
logger.info("  Batch size = %d", predict_batch_size)

In [ ]:
model.eval()
model.to(device)
all_results = []
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if n_gpu > 0:
    torch.cuda.manual_seed_all(SEED)

## KorQuAD1.0 검증

In [ ]:
logger.info("Start evaluating!")
for input_ids, input_mask, segment_ids, example_indices in tqdm(dataloader, desc="Evaluating", leave=True, position=0):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    with torch.no_grad():
        batch_start_logits, batch_end_logits = model(input_ids)
    for i, example_index in enumerate(example_indices):
        start_logits = batch_start_logits[i].detach().cpu().tolist()
        end_logits = batch_end_logits[i].detach().cpu().tolist()
        eval_feature = eval_features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        all_results.append(RawResult(unique_id=unique_id,
                                     start_logits=start_logits,
                                     end_logits=end_logits))
output_prediction_file = os.path.join(output_dir, prediction_name)
output_nbest_file = os.path.join(output_dir, nbest_name)
write_predictions(eval_examples, eval_features, all_results,
                    n_best_size, max_answer_length,
                    False, output_prediction_file, output_nbest_file,
                    None, False, False, 0.0)

## 결과 확인

In [ ]:
import json

expected_version = 'KorQuAD_v1.0'
with open(predict) as dataset_file:
    dataset_json = json.load(dataset_file)
    read_version = "_".join(dataset_json['version'].split("_")[:-1])
    if (read_version != expected_version):
        logger.info('Evaluation expects ' + expected_version +
                    ', but got dataset with ' + read_version,
                    file=sys.stderr)
    dataset = dataset_json['data']
with open('/content/drive/My Drive/Colab Notebooks/reformer/korquad/reformer-mlm_predictions_14.json') as prediction_file:
    predictions = json.load(prediction_file)
logger.info(json.dumps(evaluate(dataset, predictions)))